##### import

In [1]:
from cvzone.ColorModule import ColorFinder
import cvzone
import cv2
import numpy as np
import matplotlib.pyplot as plt

##### Kalmann Filter

In [2]:
class KalmanFilter(object):
    def __init__(self, dt, u_x,u_y, u_z, std_acc, x_std_meas, y_std_meas, z_std_meas):
        """
        :param dt: sampling time (time for 1 cycle)
        :param u_x: acceleration in x-direction
        :param u_y: acceleration in y-direction
        :param std_acc: process noise magnitude
        :param x_std_meas: standard deviation of the measurement in x-direction
        :param y_std_meas: standard deviation of the measurement in y-direction
        """

        # Define sampling time
        self.dt = dt

        # Define the  control input variables
        self.u = np.matrix([u_x,u_y,u_z])

        # Intial State [x, dx, y, dy, z, dz]
        self.x = np.matrix([[0], 
                            [0], 
                            [0], 
                            [0], 
                            [0], 
                            [0]])

        # Define the State Transition Matrix A
        self.F = np.matrix([[1, self.dt, 0, 0, 0, 0],
                            [0, 1, 0, 0, 0, 0],
                            [0, 0, 1, self.dt, 0, 0],
                            [0, 0, 0, 1, 0, 0],
                            [0, 0, 0, 0, 1, self.dt],
                            [0, 0, 0, 0, 0, 1]])

        # Define the Control Input Matrix B
        self.G = np.matrix([[(self.dt**2)/2],
                            [self.dt],
                            [(self.dt**2)/2],
                            [self.dt],
                            [(self.dt**2)/2],
                            [self.dt]])

        # Define Measurement Mapping Matrix
        self.H = np.matrix([[1, 0, 0, 0, 0, 0],
                            [0, 0, 1, 0, 0, 0],
                            [0, 0, 0, 0, 1, 0]])

        #Initial Process Noise Covariance
        self.Q = np.matrix([[(self.dt**4)/4, (self.dt**3)/2, 0, 0, 0, 0],
                            [(self.dt**3)/2, self.dt**2, 0, 0, 0, 0],
                            [0, 0, (self.dt**4)/4, (self.dt**3)/2, 0, 0],
                            [0, 0, (self.dt**3)/2, self.dt**2, 0, 0],
                            [0, 0, 0, 0, (self.dt**4)/4, (self.dt**3)/2],
                            [0, 0, 0, 0, (self.dt**3)/2, self.dt**2],]) * std_acc**2

        #Initial Measurement Noise Covariance
        self.R = np.matrix([[x_std_meas**2, 0, 0],
                           [0, y_std_meas**2, 0],
                           [0, 0, z_std_meas**2]])

        #Initial Covariance Matrix
        self.P = np.eye(self.F.shape[1])

    def predict(self):
        # Refer to :Eq.(9) and Eq.(10)  in https://machinelearningspace.com/object-tracking-simple-implementation-of-kalman-filter-in-python/?preview_id=1364&preview_nonce=52f6f1262e&preview=true&_thumbnail_id=1795

        # Update time state
        #x_k =Ax_(k-1) + Bu_(k-1)     Eq.(9)
        print(np.shape(np.dot(self.F, self.x)), np.dot(self.F, self.x))
        print(np.shape(np.dot(self.G, self.u)), np.dot(self.G, self.u))
        # self.x = np.dot(self.F, self.x) + np.dot(self.G, self.u)
        # print(np.shape(self.x), self.x)
        self.x = np.dot(self.F, self.x) # disable control input

        # Calculate error covariance
        # P= A*P*A' + Q               Eq.(10)
        self.P = np.dot(np.dot(self.F, self.P), self.F.T) + self.Q
        return (self.x[0], self.x[2], self.x[4])

    def update(self, z):

        # Refer to :Eq.(11), Eq.(12) and Eq.(13)  in https://machinelearningspace.com/object-tracking-simple-implementation-of-kalman-filter-in-python/?preview_id=1364&preview_nonce=52f6f1262e&preview=true&_thumbnail_id=1795
        # S = H*P*H'+R
        S = np.dot(self.H, np.dot(self.P, self.H.T)) + self.R

        # Calculate the Kalman Gain
        # K = P * H'* inv(H*P*H'+R)
        K = np.dot(np.dot(self.P, self.H.T), np.linalg.inv(S))  #Eq.(11)

        self.x = np.round(self.x + np.dot(K, (z - np.dot(self.H, self.x))))   #Eq.(12)

        I = np.eye(self.H.shape[1])
        # temp = I - (K * self.H)
        temp = I - (K * self.H)

        # Update error covariance matrix
        self.P = temp * self.P * np.linalg.inv(temp) + K*self.R*K.T  #Eq.(13)
        return (self.x[0], self.x[2], self.x[4])


In [3]:
class KalmanFilter1D(object):
    def __init__(self, dt, u, std_acc, std_meas):
        self.dt = dt
        self.u = u
        self.std_acc = std_acc

        self.F = np.matrix([[1, self.dt],
                            [0, 1]])
        self.G = np.matrix([[(self.dt**2)/2], 
                            [self.dt]])

        self.H = np.matrix([[1, 0]])

        self.Q = np.matrix([[(self.dt**4)/4, (self.dt**3)/2],
                            [(self.dt**3)/2, self.dt**2]]) * self.std_acc**2

        self.R = std_meas**2

        self.P = np.eye(self.F.shape[1])
        
        self.x = np.matrix([[0], 
                            [0]])

        print(self.Q)


    def predict(self):
        # Ref :Eq.(9) and Eq.(10)

        # Update time state
        self.x = np.dot(self.F, self.x) + np.dot(self.G, self.u)

        # Calculate error covariance
        # P= A*P*A' + Q
        self.P = np.dot(np.dot(self.F, self.P), self.F.T) + self.Q
        return self.x[0]

    def update(self, z):
        # Ref :Eq.(11) , Eq.(11) and Eq.(13)
        # S = H*P*H'+R
        S = np.dot(self.H, np.dot(self.P, self.H.T)) + self.R

        # Calculate the Kalman Gain
        # K = P * H'* inv(H*P*H'+R)
        K = np.dot(np.dot(self.P, self.H.T), np.linalg.inv(S))  # Eq.(11)

        self.x = np.round(self.x + np.dot(K, (z - np.dot(self.H, self.x))))  # Eq.(12)

        I = np.eye(self.H.shape[1])
        self.P = (I - (K * self.H)) * self.P  # Eq.(13)
        return self.x[0]

##### object detection + tracking

In [11]:

if(0):
     # Create opencv video capture object
    cap = cv2.VideoCapture('video/tes_jarak_bola_2.mp4')
else:
    cap = cv2.VideoCapture(0)

cap.set(3, 1280)
cap.set(4, 720)
 
myColorFinder = ColorFinder(0)
# hsvVals = {'hmin': 33, 'smin': 72, 'vmin': 126, 'hmax': 58, 'smax': 255, 'vmax': 255} # hijau
# hsvVals = {'hmin': 23, 'smin': 41, 'vmin': 126, 'hmax': 62, 'smax': 174, 'vmax': 227} # bola kuning kehijauan
hsvVals = {'hmin': 35, 'smin': 42, 'vmin': 123, 'hmax': 73, 'smax': 124, 'vmax': 255} # bola hijau tua
# hsvVals = {'hmin': 69, 'smin': 62, 'vmin': 50, 'hmax': 107, 'smax': 255, 'vmax': 117} # hijau tua
# hsvVals = {'hmin': 6, 'smin': 42, 'vmin': 53, 'hmax': 32, 'smax': 206, 'vmax': 156} # kulit

#define kernel size  
kernel = np.ones((7,7),np.uint8)

#Create KalmanFilter object KF
#KalmanFilter(dt, u_x, u_y, u_z, std_acc, x_std_meas, y_std_meas, z_std_meas)

KF = KalmanFilter(0.1, 1, 1, 1, 1, 0.1, 0.1, 0.1)
KF_X = KalmanFilter1D(0.1, 1, 1, 0.1)
KF_Y = KalmanFilter1D(0.1, 1, 1, 0.1)
KF_Z = KalmanFilter1D(0.1, 1, 1, 0.1)
try:
    while(True):
        success, img = cap.read()
        # h, w, _ = img.shape
        imageColor, mask = myColorFinder.update(img, hsvVals)
        # print("mask shape:", np.shape(mask), "value:", np.unique(mask))

        # Remove unnecessary noise from mask
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if(len(contours) > 0):
            biggestContour = max(contours, key = cv2.contourArea)
            cv2.drawContours(img, biggestContour, -1, (0,255,0), 3)

            # bisa dipake buat approximate lokasi di video depth
            # https://stackoverflow.com/questions/69637673/finding-points-within-a-contour-using-opencv
            # https://stackoverflow.com/questions/70438811/reading-frames-from-two-video-sources-is-not-in-sync-opencv
            x,y,w,h = cv2.boundingRect(biggestContour) 
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)

            # Predict
            # (x, y, z) = KF.predict()
            x, y, z = KF_X.predict(), KF_Y.predict(), KF_Z.predict()
            # print(x,y,z)
            # Draw a rectangle as the predicted object position
            cv2.rectangle(img, (int(x - 15), int(y - 15)), (int(x + 15), int(y + 15)), (0, 0, 255), 2)

            # Update
            M = cv2.moments(biggestContour)
            if(M['m00'] != 0):
                cx = int(M['m10']/M['m00'])
                cy = int(M['m01']/M['m00'])
            else:
                cx = 0
                cy = 0
            # (x1, y1, z1) = KF.update([cx, cy, 0])
            x1, y1, z1 = KF_X.update(cx), KF_Y.update(cy), KF_Z.update(0)
        else:
            # Predict
            # (x, y, z) = KF.predict()
            x, y, z = KF_X.predict(), KF_Y.predict(), KF_Z.predict()
            # print(x,y,z)

            # Update
            M = cv2.moments(biggestContour)
            if(M['m00'] != 0):
                cx = int(M['m10']/M['m00'])
                cy = int(M['m01']/M['m00'])
            else:
                cx = 0
                cy = 0
            # (x1, y1, z1) = KF.update([cx, cy, 0])
            x1, y1, z1 = KF_X.update(cx), KF_Y.update(cy), KF_Z.update(0)

            # Draw a rectangle as the predicted object position
            cv2.rectangle(img, (int(x - 15), int(y - 15)), (int(x + 15), int(y + 15)), (0, 0, 255), 2)
            cv2.rectangle(img, (int(x - 15), int(y - 15)), (int(x + 15), int(y + 15)), (0, 255, 255), 2)
        
        imageStack = cvzone.stackImages([img, mask], 2, 0.5)
        cv2.imshow("imageStack", imageStack)


        if cv2.waitKey(2) & 0xFF == ord('q'):
                cap.release()
                cv2.destroyAllWindows()
                break

        cv2.waitKey(1)
except:
    cap.release()
    cv2.destroyAllWindows()



[[2.5e-05 5.0e-04]
 [5.0e-04 1.0e-02]]
[[2.5e-05 5.0e-04]
 [5.0e-04 1.0e-02]]
[[2.5e-05 5.0e-04]
 [5.0e-04 1.0e-02]]
